In [ ]:
from trackmate_analysis.io import (
    parse_trackmate_xml,
    extract_image_metadata,
    extract_edges_from_xml,
    extract_track_filters
)
from trackmate_analysis.tracks import (
    filter_spots_in_tracks,
    filter_spots_by_duration,
    filter_tracks_by_duration,
)
from trackmate_analysis.lineage import (
    detect_split_events, promote_split_children,
    build_track_lineage_from_splits,
)
from trackmate_analysis.plotting import (
    plot_trajectories, plot_track_histograms,
    plot_features_over_time,
)

import matplotlib.pyplot as plt

In [ ]:
xml_file = r"YOUR_XML_FILE"

meta = extract_image_metadata(xml_file)
print(meta)

spots_df, track_stats = parse_trackmate_xml(xml_file)

In [ ]:
track_stats.head()

In [ ]:
# Access individual metadata
print("Image:", meta["filename"])
print("Frames:", meta["nframes"])
print("Slices:", meta["nslices"])
print("Pixel size:", meta["pixelwidth"], "x", meta["pixelheight"])
print("Voxel depth:", meta["voxeldepth"])
print("Image width:", meta["width"])
print("Image height:", meta["height"])
print("Time interval:", meta["timeinterval"])

In [ ]:
# Plot
plot_trajectories(spots_df, width=meta.get("width"), height=meta.get("height"))
plot_track_histograms(track_stats)

In [ ]:
# Track filters
filters_df = extract_track_filters(xml_file)
display(filters_df)

In [ ]:
# Filter
spots_df = filter_spots_in_tracks(spots_df, track_stats)
spots_df = filter_spots_by_duration(spots_df, track_stats, min_duration=4, max_duration=None)
track_stats = filter_tracks_by_duration(track_stats, min_duration=4, max_duration=None)

In [ ]:
# Plot clean
plot_trajectories(spots_df, width=meta.get("width"), height=meta.get("height"))
plot_track_histograms(track_stats)

In [ ]:
# Lineage & splits
edges_df = extract_edges_from_xml(xml_file)
splits_df = detect_split_events(spots_df, edges_df)
spots_df = promote_split_children(spots_df, edges_df)
G, splits_df = build_track_lineage_from_splits(spots_df, edges_df)

In [ ]:
# Features over time
fig, axs = plot_features_over_time(
    spots_df, track_id=1,
    features=["CIRCULARITY", "MEAN_INTENSITY_CH2"],
    lineage_graph=G,
    nframes=int(meta.get("nframes", 0)),
)
plt.show()

In [ ]:
# Loop over all tracks
for tid in spots_df.loc[~spots_df["is_pseudo"], "track_id"].dropna().unique():
    fig, axs = plot_features_over_time(
        spots_df,
        track_id=int(tid),
        features=["MEAN_INTENSITY_CH2", "CIRCULARITY"],
        lineage_graph=G,
        nframes=int(meta.get("nframes", 0)),
    )
    plt.show()